In [5]:
!rm -rf Eunomia
!git clone https://github.com/ouaguenouni/Eunomia
!pip install -r Eunomia/requirements.txt

Cloning into 'Eunomia'...
remote: Enumerating objects: 97, done.
remote: Counting objects: 100% (97/97), done.
remote: Compressing objects: 100% (71/71), done.
remote: Total 97 (delta 47), reused 72 (delta 22), pack-reused 0
Receiving objects: 100% (97/97), 57.05 KiB | 2.19 MiB/s, done.
Resolving deltas: 100% (47/47), done.


In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from Eunomia.preferences import *
from Eunomia.additive_functions import *
from Eunomia.alternatives import *
from Eunomia.sampling import *
from Eunomia.mcmc import *
from Eunomia.degree import *
from Eunomia.experiments import *
pyro.set_rng_seed(1)

In [ ]:
import time

TEST_NAME = "SX1"
REPLACE = False

n_vals = [5,6,7,8,9,10]
sigma_weights = 10
sigma_w_vals = [1, 1e-1, 1e-2, 1e1]
sigma_p_vals = [1, 1e-1, 1e-2, 1e1]
n_samples = 1500


for n in n_vals:
    for m in list(2**i for i in range(2, n)):
        for k in range(1, n-1):
                for sigma_w in sigma_w_vals:
                    for sigma_p in sigma_p_vals:

                        run_d = {
                            "n": n,
                            "k": k,
                            "m": m,
                            "sigma_w": sigma_w,
                            "sigma_p": sigma_p,
                            "n_samples": n_samples
                        }

                        found = find_experiment_file(run_d, TEST_NAME)

                        if found:
                            date = found
                            print(f"Found file: {file_name}")

                            if REPLACE:
                                print('Replacing it...')
                            else:
                                continue
                        else:
                            file_name = compute_experiment_file_name(run_d, TEST_NAME)
                            print("Registering the results in: ", file_name)

                        theta = generate_additive_theta(n, k)
                        weights = generate_normal_weights(theta, sigma_weights)
                        alternatives = generate_random_alternatives_matrix(m, n)
                        ranks = compute_ws_ranks(alternatives, theta, weights)
                        t_sv = compute_semivalues(n, theta, weights, lambda x:1)
                        preferences = PreferenceModel(alternatives, ranks)
                        data = preferences.generate_preference_matrix(theta)
                        data = torch.tensor(data).float()
                        t = time.time()
                        model = posterior_sampling_model(data, sigma_w=sigma_w, sigma_p=sigma_p)
                        diag, sampled_weights, sigmas = sample_model(model, data, "w", "sigma", warmup_steps=500, num_samples=n_samples, return_diag=True)
                        t = time.time() - t
                        accs_d = get_acc_distribution(data, sampled_weights, sigmas)

                        predicted_rankings = [np.argsort(compute_semivalues(n, theta, weights, lambda x:1))[::-1] for weights in sampled_weights]
                        kt_d = get_kt_distribution(predicted_rankings, np.argsort(t_sv))

                        file_path = record_experiment_results(run_d, TEST_NAME)
                        run_d["time"] = t
                        run_d["weights"] = weights.tolist()
                        run_d["predicted_rankings"] = [i.tolist() for i in predicted_rankings]
                        run_d["accuracy_distribution"] = [i.tolist() for i in accs_d]
                        run_d["kt_d"] = [i.tolist() for i in kt_d]
                        run_d["acceptance_rate"] = diag["acceptance rate"]["chain 0"]
                        run_d["w_eff"] = [float(i) for i in diag["w"]["n_eff"].numpy()]
                        run_d["w_rhat"] = [float(i) for i in diag["w"]["r_hat"].numpy()]

                        print("diag: ", diag)
                        with open(file_path, 'w') as file:
                            yaml.dump(run_d, file, default_flow_style=False)



Registering the results in:  SX1/2023-11-18/n-5_k-1_m-4_sigma_w-1_sigma_p-1_n_samples-1500.yaml


Sample: 100%|████████████████████████████| 2000/2000 [01:36, 20.62it/s, step size=1.70e-01, acc. prob=0.907]


diag:  {'sigma': OrderedDict([('n_eff', tensor(821.4919)), ('r_hat', tensor(0.9998))]), 'w': OrderedDict([('n_eff', tensor([810.8021, 823.4952, 878.3043, 877.1406, 565.9174])), ('r_hat', tensor([1.0001, 0.9994, 1.0039, 1.0000, 0.9997]))]), 'divergences': {'chain 0': [175, 501, 558, 662, 665, 666, 677, 680, 685, 780, 789, 1213, 1478]}, 'acceptance rate': {'chain 0': 0.9933333333333333}}
Registering the results in:  SX1/2023-11-18/n-5_k-1_m-4_sigma_w-1_sigma_p-0.1_n_samples-1500.yaml


Sample: 100%|████████████████████████████| 2000/2000 [00:48, 41.51it/s, step size=3.72e-01, acc. prob=0.895]


diag:  {'sigma': OrderedDict([('n_eff', tensor(910.2490)), ('r_hat', tensor(1.0024))]), 'w': OrderedDict([('n_eff', tensor([ 962.2154,  914.4397, 1273.5034, 1261.7496, 1054.6635])), ('r_hat', tensor([0.9994, 1.0040, 0.9994, 1.0019, 0.9996]))]), 'divergences': {'chain 0': [6, 8, 57, 701, 801, 1155, 1157, 1160, 1162, 1172, 1389, 1459]}, 'acceptance rate': {'chain 0': 0.978}}
Registering the results in:  SX1/2023-11-18/n-5_k-1_m-4_sigma_w-1_sigma_p-0.01_n_samples-1500.yaml


...

Sample: 100%|████████████████████████████| 2000/2000 [00:49, 40.59it/s, step size=2.10e-01, acc. prob=0.829]


diag:  {'sigma': OrderedDict([('n_eff', tensor(1049.6807)), ('r_hat', tensor(1.0001))]), 'w': OrderedDict([('n_eff', tensor([ 702.1335, 1217.0377,  837.6376, 1237.5428, 1201.3760, 1301.0325,
         837.8002,  887.8880,  692.5925, 1282.6327,  910.5652,  843.5082,
        1181.5840,  781.0720,  975.6763])), ('r_hat', tensor([0.9994, 0.9998, 0.9995, 0.9996, 0.9996, 0.9996, 0.9994, 1.0001, 0.9993,
        0.9998, 0.9995, 1.0000, 1.0019, 0.9999, 0.9994]))]), 'divergences': {'chain 0': [81, 96, 100, 225, 313, 446, 672, 850, 853, 893, 915, 926, 927, 930, 1053, 1090, 1247, 1291, 1424, 1440]}, 'acceptance rate': {'chain 0': 0.9846666666666667}}
Registering the results in:  SX1/2023-11-18/n-5_k-2_m-4_sigma_w-10.0_sigma_p-0.01_n_samples-1500.yaml


Sample: 100%|████████████████████████████| 2000/2000 [00:30, 65.04it/s, step size=3.96e-01, acc. prob=0.782]


diag:  {'sigma': OrderedDict([('n_eff', tensor(656.1917)), ('r_hat', tensor(0.9994))]), 'w': OrderedDict([('n_eff', tensor([ 733.6885, 1120.6742,  774.5565, 1339.6721, 1125.5956, 1026.6742,
         462.8588,  844.9200, 1059.4149,  627.3074, 1128.7629,  820.2537,
        1529.3389,  918.3757,  881.6891])), ('r_hat', tensor([1.0009, 0.9993, 1.0029, 0.9997, 1.0010, 0.9996, 1.0022, 0.9995, 0.9994,
        1.0030, 1.0025, 0.9997, 1.0008, 1.0064, 1.0008]))]), 'divergences': {'chain 0': [0, 43, 51, 52, 82, 86, 89, 91, 102, 123, 149, 152, 212, 271, 275, 376, 412, 415, 416, 438, 493, 500, 524, 550, 558, 680, 681, 682, 725, 730, 731, 732, 751, 753, 799, 864, 869, 870, 871, 873, 896, 900, 906, 1008, 1009, 1010, 1011, 1012, 1016, 1018, 1044, 1082, 1103, 1108, 1137, 1226, 1227, 1246, 1260, 1261, 1279, 1293, 1318, 1385, 1393, 1399, 1400, 1402, 1444, 1474, 1490, 1497]}, 'acceptance rate': {'chain 0': 0.9626666666666667}}
Registering the results in:  SX1/2023-11-18/n-5_k-2_m-4_sigma_w-10.0_sigma_p-10

Sample:  71%|███████████████████▉        | 1424/2000 [21:16,  1.03s/it, step size=1.79e-03, acc. prob=0.927]